# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [ ]:
!pip install "pymysql==1.0.2"

In [2]:
!pip install sqlalchemy --upgrade

In [3]:
!pip install mysql-connector-python

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

host = 'database-1-arv-borrar-clase.cb4kwk2a468v.eu-north-1.rds.amazonaws.com'
username = 'admin'
# password = "12345678"
password = os.getenv('AWS_PASSWORD')
port = 3306

### Conexion BD

In [2]:
import pymysql

In [3]:
'''
pymysq].cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db. cursor()

OperationalError: (2003, "Can't connect to MySQL server on 'database-1-arv-borrar-clase.cb4kwk2a468v.eu-north-1.rds.amazonaws.com' ([Errno 8] nodename nor servname provided, or not known)")

### Version DB

In [ ]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc '''
cursor.execute("SELECT VERSION()")
version = cursor.fetchone ()
print (f"MysQL version: {version}")

MysQL version: {'VERSION()': '8.0.39'}


### Creación de DB

In [ ]:
#Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor.execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

ProgrammingError: (1007, "Can't create database 'country_database'; database exists")

In [ ]:
# Podemos eliminar la BD
#drop_db = '''DROP DATABASE country_database'''
#cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas


1

In [ ]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [ ]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

OperationalError: (1046, 'No database selected')

### Seleccionar la BD

In [ ]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

0

In [ ]:
# Chequear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [ ]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

### Insertar datos

In [ ]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe',47, 1.74)
'''
cursor.execute(insert_data)

1

In [ ]:
name = "France"
continent = "Europe"
population = 54
gdp = 2

In [ ]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})'''

In [ ]:
cursor.execute(insert_data)

1

### Leer datos

In [ ]:
query = '''SELECT * FROM country WHERE name LIKE "%pa%"'''
cursor.execute(query)

1

In [ ]:
print(cursor.fetchall())

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 47, 'gdp': 1.74}]


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [ ]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [ ]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [ ]:
# Creamos el CSV
import pandas as pd

df= pd.read_csv('country_data.csv', sep=';')

In [ ]:
df

,name,continent,population,gdp
0,France,Europe,70,2.7
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.8


In [ ]:
for datos in zip(*[valor.values() for clave,valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s','%s',%d,%.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [ ]:
query = "SELECT * FROM country"
cursor.execute(query)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 54,
  'gdp': 2.0},
 {'id': 3,
  'name': 'France',
  'continent': 'Europe',
  'population': 70,
  'gdp': 2.7},
 {'id': 4,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 126,
  'gdp': 4.9},
 {'id': 5,
  'name': 'Argentina',
  'continent': 'America',
  'population': 44,
  'gdp': 0.5},
 {'id': 6,
  'name': 'Brasil',
  'continent': 'America',
  'population': 209,
  'gdp': 1.8}]

In [ ]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine
import pymysql

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = "country_database"))

In [ ]:
df.to_sql(name = "country", con= engine, if_exists= "append", index = False)

4

In [ ]:
db.commit()

### Cerrar la conexión cuando acabemos

In [ ]:
db.close()

In [ ]:
conn = engine.connect()
conn.close()